# ZFS - Truly 128-bit and counting.

## - or -   

# Investiagtions in ZFS

### Check the release of this particular CentOS

In [ ]:
cat /etc/redhat-release

### Once verified pick the repository that is correct for the system

In [ ]:
yum -y install http://download.zfsonlinux.org/epel/zfs-release.el7_4.noarch.rpm

### Choose wisely DKMS vs KMOD
DKMS - bespoke and takes advantage of the Dynamic Kernel Module System excellent for systems that are customized.

KMOD - kABI-tracking, pre-built per kernel version and fewer dependancies

Once chosen investigate the repository to alter the enabled of either DKMS or KMOD as needed. Investigate with the cell below.

In [ ]:
cat /etc/yum.repos.d/zfs.repo

### Once verified pick the repository that is correct for the system and alter to `enabled=1` AND `enabled=0` as needed.

In [ ]:
yum -y install yum-utils
yum-config-manager --disable zfs
yum-config-manager --enable zfs-kmod

### Lets install

In [ ]:
yum -y install zfs

### Ensure that the module is loaded

In [ ]:
modprobe zfs && zpool status

### Ensure that at reboot we'll get the pools remounted and healthy by showing systemd some love

In [ ]:
systemctl preset zfs-import-cache zfs-import-scan zfs-import.target zfs-mount zfs-share zfs-zed zfs.target

# We are ready to create the pool
## Start by listing out the block devices on the system (we want sda and sdb)

In [ ]:
lsblk -f

## Now create a mirrored pool called `tank` that is constructed from the block devices `sda` and `sdb`

In [ ]:
zpool create tank -f mirror sda sdb && zpool status

## Disks can get shuffled and sda or sdb can re-letter in the worst times so lets alter to disks by-id.  Investigate the differences in the zpool status output from above.

In [ ]:
zpool export tank && zpool import tank -d /dev/disk/by-id && zpool status

## Note that we do not need to alter `/etc/fstab` for any reason

In [ ]:
df -Th

## Lets destroy the mirror and build out a stripe (we want to go fast).  

## NOTE: the && is used to seperate commands on a single line.

In [ ]:
zpool destroy tank && zpool create tank -f sda sdb && zpool export tank && zpool import tank -d /dev/disk/by-id && zpool status

## Notice the size is doubled and that the pools overall IOPS capacity will have doubled too.

In [ ]:
df -Th